In [1]:
import logging
import threading
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.db import Database
from iotfunctions.enginelog import EngineLogging
from iotfunctions import estimator
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.enginelog import EngineLogging
from iotfunctions import pipeline as pp
from iotfunctions.pipeline import Db2DataWriter

import datetime as dt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

/home/markus/.local/lib/python3.7/site-packages/iotfunctions/bif.py:1946: UserWarning: IoTCalcSettings is deprecated. Use entity type constants instead of a metadata provider to set entity type properties
  warnings.warn(('IoTCalcSettings is deprecated. Use entity type constants'


In [2]:
credentials = {
  "tenantId": "AnalyticsServiceDev",
  "as_api_host": "https://api-dev.connectedproducts.internetofthings.ibmcloud.com",
  "as_api_key": "a-69xgm4-8bdgtvnsv4",
  "as_api_token": "9X_tMKdupOiJ!mzaPV",
  "config" : {
      "objectStorageEndpoint" : "https://s3-api.us-geo.objectstorage.softlayer.net",
      "bos_runtime_bucket" : "analytics-runtime-analyticsservicedev-799d2008b460",
      "bos_logs_bucket" : "analytics-logs-analyticsservicedev-32703c52ec8b"
  },
  "objectStorage": {
      "username" : "58ddd86b5de8468b819d385046f17033",
      "password" : "ee0d6c5521ce9ff100f91b0e37d4eb8cc1a038b5a6d05b38",
      "region" : "us",
      "endpoint" : "https://s3-api.us-geo.objectstorage.softlayer.net"
  },
  "db2": {
    "username": "bluadmin",
    "password": "ZmM5MmE5NmZkZGZl",
    "databaseName": "BLUDB",
    "port": 50000,
    "httpsUrl": "https://dashdb-enterprise-yp-dal13-74.services.dal.bluemix.net:50000",
    "host": "dashdb-enterprise-yp-dal13-74.services.dal.bluemix.net"
  },
  "postgres": None
}
EngineLogging.configure_console_logging(logging.DEBUG)

2019-09-18T21:18:42.932 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10


In [3]:
db_schema = None
db = Database(credentials=credentials)
print (db)
# extend iotfunctions/db.py to support postgres connection string in credentials

2019-09-18T21:18:44.693 DEBUG iotfunctions.db.__init__ Unable to locate message_hub credentials. Database object created, but it will not be able interact with message hub.
2019-09-18T21:18:44.695 DEBUG iotfunctions.db.__init__ created a CosClient object
2019-09-18T21:18:47.504 DEBUG iotfunctions.db.__init__ Db connection established
2019-09-18T21:18:47.507 DEBUG urllib3.connectionpool._new_conn Starting new HTTPS connection (1): api-dev.connectedproducts.internetofthings.ibmcloud.com


/home/markus/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-09-18T21:18:51.171 DEBUG urllib3.connectionpool._make_request https://api-dev.connectedproducts.internetofthings.ibmcloud.com:443 "GET /api/meta/v1/AnalyticsServiceDev/entityType HTTP/1.1" 200 None
2019-09-18T21:18:51.692 DEBUG iotfunctions.db.http_request http request successful. status 200
Database object:
  Connection: db2+ibm_db://bluadmin:ZmM5MmE5NmZkZGZl@dashdb-enterprise-yp-dal13-74.services.dal.bluemix.net:50000/BLUDB;


In [4]:
table = db.get_table("MARKUS_TEST_ROBOT1")
start_ts = dt.datetime.utcnow() - dt.timedelta(days=30)
end_ts = dt.datetime.utcnow()
df = db.read_table(table, None, None, None, "evt_timestamp", start_ts, end_ts)
print (table)

/home/markus/.local/lib/python3.7/site-packages/sqlalchemy/engine/reflection.py:913: SAWarning: index key 'sqlnotapplicable' was not located in columns for table 'MARKUS_TEST_ROBOT1'
  "columns for table '%s'" % (flavor, c, table_name)


MARKUS_TEST_ROBOT1


In [6]:
print (df.head(10))

  deviceid              evt_timestamp          devicetype logicalinterface_id  \
0    73002 2019-08-19 17:12:46.947824  Markus_Test_Robot1                       
1    73001 2019-08-19 17:12:36.690739  Markus_Test_Robot1                       
2    73001 2019-08-19 17:17:36.690739  Markus_Test_Robot1                       
3    73001 2019-08-19 17:17:39.128473  Markus_Test_Robot1                       
4    73003 2019-08-19 17:22:39.128473  Markus_Test_Robot1                       
5    73000 2019-08-19 17:27:53.336565  Markus_Test_Robot1                       
6    73004 2019-08-19 17:32:53.336565  Markus_Test_Robot1                       
7    73002 2019-08-19 17:33:00.612067  Markus_Test_Robot1                       
8    73002 2019-08-19 17:38:00.612067  Markus_Test_Robot1                       
9    73002 2019-08-19 17:37:39.072851  Markus_Test_Robot1                       

  eventtype format updated_utc plant_code     torque  tool_type        load  \
0        tn               Non

In [4]:
# Generate 5 mins of data in table 'testdata' with a single additional column of TestData
EngineLogging.configure_console_logging(logging.DEBUG)
jobsettings = {'_timestamp' : 'TIMESTAMP'}
et = metadata.EntityType('markus_testdata', db, 
                         bif.EntityDataGenerator(output_item='my_test_gen'),
                         Column('TestData',Float()),
                         **jobsettings)

#start_date = dt.datetime.utcnow() - dt.timedelta(days=1)
#et.exec_local_pipeline(start_ts = start_date)
df = et.generate_data()

2019-09-18T21:18:58.232 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10
2019-09-18T21:18:58.235 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 2.0.3
2019-09-18T21:18:58.237 DEBUG iotfunctions.pipeline.__init__ Starting trace
2019-09-18T21:18:58.238 DEBUG iotfunctions.pipeline.__init__ Trace name: auto_trace_markus_testdata_20190918191858
2019-09-18T21:18:58.240 DEBUG iotfunctions.pipeline.__init__ auto_save None
2019-09-18T21:18:58.241 WARNING iotfunctions.metadata.__init__ No _db_schema specified in **kwargs. Usingdefault database schema.
2019-09-18T21:18:58.243 DEBUG iotfunctions.util.categorize_args categorizing arguments


/home/markus/.local/lib/python3.7/site-packages/sqlalchemy/engine/reflection.py:913: SAWarning: index key 'sqlnotapplicable' was not located in columns for table 'markus_testdata'
  "columns for table '%s'" % (flavor, c, table_name)


2019-09-18T21:19:02.208 DEBUG iotfunctions.metadata.__init__ Initialized entity type 
EntityType:markus_testdata
Functions:
EntityDataGenerator at granularity None required inputs not evaluated yet outputs produced not evaluated yet on schedule None
Granularities:
No schedules metadata
2019-09-18T21:19:02.210 DEBUG iotfunctions.metadata.generate_data Generating data for markus_testdata with metrics ['TestData'] and dimensions ['eventtype'] and dates []
2019-09-18T21:19:02.236 DEBUG iotfunctions.automation.get_data Generated 6 rows of time series data from 2019-09-18 19:14:02.212462 to 2019-09-18 19:19:02.212462
2019-09-18T21:19:03.694 INFO iotfunctions.db.write_frame Wrote data to table markus_testdata 


In [12]:
from iotfunctions import pipeline as pp
job = pp.JobController(et)
job.execute()

TBD ***** - Add stages for usage stats and write to MessageHub


/home/markus/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [10]:
print (df)

   TestData deviceid                  timestamp entity_id eventtype  \
0 -0.442028    73004 2019-09-18 17:08:07.081637        tt        te   
1  1.459966    73002 2019-09-18 17:09:07.081637        ny        ee   
2  1.521797    73001 2019-09-18 17:10:07.081637        yn        en   
3  0.432316    73001 2019-09-18 17:11:07.081637        ty        ee   
4  2.985946    73004 2019-09-18 17:12:07.081637        ei        ve   
5  1.247824    73002 2019-09-18 17:13:07.081637        td        vp   

  logicalinterface_id       devicetype format updated_utc  
0                      markus_testdata               None  
1                      markus_testdata               None  
2                      markus_testdata               None  
3                      markus_testdata               None  
4                      markus_testdata               None  
5                      markus_testdata               None  


In [84]:
# read it back
table = db.get_table("testdata")
start_ts = dt.datetime.utcnow() - dt.timedelta(days=1)
end_ts = dt.datetime.utcnow()
df_in = db.read_table(table, None, None, None, "evt_timestamp", start_ts, end_ts)
print (table)
print (df_in.shape)

testdata
(281, 8)


In [85]:
print (df_in.tail(5))

    deviceid              evt_timestamp devicetype logicalinterface_id  \
276    73000 2019-09-18 14:15:46.843620   testdata                       
277    73001 2019-09-18 14:16:46.843620   testdata                       
278    73003 2019-09-18 14:17:46.843620   testdata                       
279    73002 2019-09-18 14:18:46.843620   testdata                       
280    73002 2019-09-18 14:19:46.843620   testdata                       

    eventtype format updated_utc  TestData  
276        tn               None  0.790283  
277        ne               None -0.032254  
278        nn               None  0.501532  
279        ee               None  0.781938  
280        pn               None  1.688854  


In [86]:
# check whether the data for the last 5 minutes is the same - must return True
print (np.array_equal(df['TestData'].tail(5), df_in['TestData'].tail(5)))

True


In [107]:
print (et._functions)

In [6]:
import ibm_db
EngineLogging.configure_console_logging(logging.DEBUG)
et._functions = [bif.PythonExpression('5*df["TestData"]','TestOut')]
#db3 = getattr(et, 'db')
#print (db3)

db_conn = ibm_db.connect('DATABASE=BLUDB;HOSTNAME=dashdb-enterprise-yp-dal13-74.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=bluadmin;PWD=ZmM5MmE5NmZkZGZl;','','')
print (db_conn)
stmt = ibm_db.prepare(db_conn, 'SELECT * FROM "BLUADMIN"."MARKUS_TESTDATA" WHERE TIMESTAMP >=  ?  AND TIMESTAMP <  ? ')
stmt_tuple = (dt.datetime.utcnow() - dt.timedelta(days=1), dt.datetime.utcnow())
ibm_db.execute(stmt, stmt_tuple)
print (ibm_db.stmt_errormsg(stmt))
row=ibm_db.fetch_tuple(stmt)
print (row)
#ibm_db.close(db_conn)
jobsettings = {'writer_name' : Db2DataWriter, 'db_connection' : db_conn, '_db_schema': 'BLUADMIN', 'save_trace_to_file' : True}
job = pp.JobController(et, **jobsettings)
job.execute()
#start_date = dt.datetime.utcnow() - dt.timedelta(days=1)
#jobsettings = {'data_writer': Db2DataWriter, 'db_connection' : db}
#df2 = et.exec_local_pipeline(start_ts = start_date, **job_settings)

2019-09-18T21:19:51.184 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10
2019-09-18T21:19:51.186 DEBUG iotfunctions.base.parse_expression expression (5*df["TestData"])

('73000', datetime.datetime(2019, 9, 18, 17, 49, 33, 728536), 'markus_testdata', '', 'ny', '', None, 0.0014821722656370995)
2019-09-18T21:19:52.292 DEBUG iotfunctions.pipeline.set_payload_params Setting param writer_name on payload to <class 'iotfunctions.pipeline.Db2DataWriter'>
2019-09-18T21:19:52.293 DEBUG iotfunctions.pipeline.set_payload_params Setting param db_connection on payload to <ibm_db.IBM_DBConnection object at 0x7fcdfd284120>
2019-09-18T21:19:52.295 DEBUG iotfunctions.pipeline.set_payload_params Setting param _db_schema on payload to BLUADMIN
2019-09-18T21:19:52.296 DEBUG iotfunctions.pipeline.set_payload_params Setting param save_trace_to_file on payload to True
2019-09-18T21:19:52.298 DEBUG iotfunctions.pipeline.set_payload_params Setting param tenan

/home/markus/.local/lib/python3.7/site-packages/sqlalchemy/engine/reflection.py:913: SAWarning: index key 'sqlnotapplicable' was not located in columns for table 'markus_testdata'
  "columns for table '%s'" % (flavor, c, table_name)


2019-09-18T21:19:58.574 DEBUG iotfunctions.metadata.index_df Indexed dataframe on id, TIMESTAMP
2019-09-18T21:19:58.577 DEBUG iotfunctions.pipeline.exec_stage_method No method get_column_map on read_entity_data returning default None. 'DataReader' object has no attribute 'get_column_map'
2019-09-18T21:19:58.579 DEBUG iotfunctions.pipeline.merge_dataframe Merging dataframe with columns [] and index []
2019-09-18T21:19:58.582 DEBUG iotfunctions.metadata.index_df Found existing index on id, TIMESTAMP.No need to recreate index
2019-09-18T21:19:58.587 INFO iotfunctions.pipeline.update_last_entry Trace message:  Completed stage.
2019-09-18T21:19:58.588 INFO iotfunctions.pipeline.update_last_entry Trace payload: {'produces_output_items': True, 'output_items': ['deviceid', 'TIMESTAMP', 'TestData'], 'discard_prior_data': False, 'merge_result': 'existing empty df with new DataFrame', 'usage': 72, 'index': ['id', 'TIMESTAMP'], 'can_proceed': True, 'updated': datetime.datetime(2019, 9, 18, 19, 19,

/home/markus/.local/lib/python3.7/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2019-09-18T21:19:59.889 DEBUG urllib3.connectionpool._make_request https://s3-api.us-geo.objectstorage.softlayer.net:443 "PUT /analytics-runtime-analyticsservicedev-799d2008b460/AnalyticsServiceDev/markus_testdata/20190918/markus_testdata_trace_191953 HTTP/1.1" 200 0
2019-09-18T21:19:59.892 DEBUG iotfunctions.pipeline.save Saved trace to cos AnalyticsServiceDev/markus_testdata/20190918/markus_testdata_trace_191953
2019-09-18T21:19:59.894 DEBUG iotfunctions.pipeline.save wrote trace to file auto_trace_markus_testdata_20190918191953.json
2019-09-18T21:20:00.544 DEBUG iotfunctions.pipeline.update Updated job log (markus_testdata,5min): 2019-09-18 19:19:53.107124
2019-09-18T21:20:00.546 DEBUG iotfunctions.pipeline.get_next_future_execution Next scheduled execution date is 2019-09-18 19:24:53.107124
2019-09-18T21:20:00.549 DEBUG iotfunctions.pipeline.execute Ending job normally as there are no scheduled executions  due before execution end time
2019-09-18T21:20:00.644 DEBUG iotfunctions.pip

In [7]:
print (job)



Default schedule 5min 
    Schedule 5min start None:None backtracks: None 
Stages of type: get_data at grain None: 
   System generated DataReader stage: read_entity_data. Reads data from objects: markus_testdata
Stages of type: transform at grain None: 
   PythonExpression at granularity None requires inputs {'TestData'} produces outputs ['TestOut'] on schedule 5min




In [8]:
print (et.get_data())

2019-09-18T22:08:02.261 DEBUG iotfunctions.metadata.index_df Indexed dataframe on id, TIMESTAMP
                                 deviceid       devicetype  \
id    TIMESTAMP                                              
73000 2019-09-18 17:49:33.728536    73000  markus_testdata   
73003 2019-09-18 17:50:33.728536    73003  markus_testdata   
      2019-09-18 17:51:33.728536    73003  markus_testdata   
73002 2019-09-18 17:52:33.728536    73002  markus_testdata   
73004 2019-09-18 17:53:33.728536    73004  markus_testdata   
73003 2019-09-18 17:54:33.728536    73003  markus_testdata   
73002 2019-09-18 17:54:31.762757    73002  markus_testdata   
73000 2019-09-18 17:55:31.762757    73000  markus_testdata   
73001 2019-09-18 17:56:31.762757    73001  markus_testdata   
      2019-09-18 17:57:31.762757    73001  markus_testdata   
73003 2019-09-18 17:58:31.762757    73003  markus_testdata   
73004 2019-09-18 17:59:31.762757    73004  markus_testdata   
73001 2019-09-18 18:01:27.988642    